In [34]:
pip install scikeras

In [35]:
pip install keras==2.12.0

In [37]:
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from sklearn.model_selection import GridSearchCV
from scikeras.wrappers import KerasClassifier, KerasRegressor
from keras.wrappers.scikit_learn import KerasClassifier
import random
import numpy as np

In [38]:
random_seed = 73
np.random.seed(random_seed)
tf.random.set_seed(random_seed)
random.seed(random_seed)

In [39]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0
x_train = x_train.reshape((-1, 28, 28, 1))
x_test = x_test.reshape((-1, 28, 28, 1))

In [40]:
#Architecture
def create_model(neurons_per_layer=64, activation_function='relu'):
    model = Sequential([
        Flatten(input_shape=(28, 28, 1)),
        Dense(neurons_per_layer, activation=activation_function),
        Dense(10, activation='softmax')
    ])
    model.compile(optimizer='adam',
                  loss=tf.keras.losses.SparseCategoricalCrossentropy(),
                  metrics=['accuracy'])
    return model

In [41]:
model = KerasClassifier(build_fn=create_model, verbose=0)
param_grid = {
    'neurons_per_layer': [32, 64, 128,256],
    'activation_function': ['relu', 'tanh', 'sigmoid']
}
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=3, verbose=2)
grid_result = grid_search.fit(x_train, y_train)


<ipython-input-41-97123dfc420f>:2: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  model = KerasClassifier(build_fn=create_model, verbose=0)


Fitting 3 folds for each of 12 candidates, totalling 36 fits
[CV] END .....activation_function=relu, neurons_per_layer=32; total time=   5.0s
[CV] END .....activation_function=relu, neurons_per_layer=32; total time=   5.4s
[CV] END .....activation_function=relu, neurons_per_layer=32; total time=   5.6s
[CV] END .....activation_function=relu, neurons_per_layer=64; total time=   5.2s
[CV] END .....activation_function=relu, neurons_per_layer=64; total time=   7.6s
[CV] END .....activation_function=relu, neurons_per_layer=64; total time=   5.2s
[CV] END ....activation_function=relu, neurons_per_layer=128; total time=   7.4s
[CV] END ....activation_function=relu, neurons_per_layer=128; total time=   7.7s
[CV] END ....activation_function=relu, neurons_per_layer=128; total time=   6.1s
[CV] END ....activation_function=relu, neurons_per_layer=256; total time=  12.8s
[CV] END ....activation_function=relu, neurons_per_layer=256; total time=  12.6s
[CV] END ....activation_function=relu, neurons_p

In [44]:
print("Overall Results:")
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))


Overall Results:
Best: 0.951883 using {'activation_function': 'relu', 'neurons_per_layer': 256}
0.925867 (0.005400) with: {'activation_function': 'relu', 'neurons_per_layer': 32}
0.936767 (0.003718) with: {'activation_function': 'relu', 'neurons_per_layer': 64}
0.945133 (0.001992) with: {'activation_function': 'relu', 'neurons_per_layer': 128}
0.951883 (0.001299) with: {'activation_function': 'relu', 'neurons_per_layer': 256}
0.921500 (0.001056) with: {'activation_function': 'tanh', 'neurons_per_layer': 32}
0.930183 (0.000660) with: {'activation_function': 'tanh', 'neurons_per_layer': 64}
0.935700 (0.001724) with: {'activation_function': 'tanh', 'neurons_per_layer': 128}
0.937667 (0.000900) with: {'activation_function': 'tanh', 'neurons_per_layer': 256}
0.905850 (0.002117) with: {'activation_function': 'sigmoid', 'neurons_per_layer': 32}
0.914533 (0.002030) with: {'activation_function': 'sigmoid', 'neurons_per_layer': 64}
0.919650 (0.002446) with: {'activation_function': 'sigmoid', 'ne

In [43]:
activation_functions = ['relu', 'tanh', 'sigmoid']
for activation_function in activation_functions:
    activation_results = [(mean, std, param) for mean, std, param in zip(means, stds, params) if param['activation_function'] == activation_function]
    print(f"\nResults for {activation_function.capitalize()} activation:")
    for mean, std, param in activation_results:
        print("%f (%f) with: %r" % (mean, std, param))

test_acc = grid_search.score(x_test, y_test)
print('\nTest accuracy of the best model:', test_acc)


Results for Relu activation:
0.925867 (0.005400) with: {'activation_function': 'relu', 'neurons_per_layer': 32}
0.936767 (0.003718) with: {'activation_function': 'relu', 'neurons_per_layer': 64}
0.945133 (0.001992) with: {'activation_function': 'relu', 'neurons_per_layer': 128}
0.951883 (0.001299) with: {'activation_function': 'relu', 'neurons_per_layer': 256}

Results for Tanh activation:
0.921500 (0.001056) with: {'activation_function': 'tanh', 'neurons_per_layer': 32}
0.930183 (0.000660) with: {'activation_function': 'tanh', 'neurons_per_layer': 64}
0.935700 (0.001724) with: {'activation_function': 'tanh', 'neurons_per_layer': 128}
0.937667 (0.000900) with: {'activation_function': 'tanh', 'neurons_per_layer': 256}

Results for Sigmoid activation:
0.905850 (0.002117) with: {'activation_function': 'sigmoid', 'neurons_per_layer': 32}
0.914533 (0.002030) with: {'activation_function': 'sigmoid', 'neurons_per_layer': 64}
0.919650 (0.002446) with: {'activation_function': 'sigmoid', 'neuro